# KNN

In [522]:
df

,price,bedrooms,bathrooms,surface,sqft_lot,floors,waterfront,view,condition,grade,...,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,0,0,1,0,0,0
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,...,0,0,0,0,0,0,0,0,0,0
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21081,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
21082,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,...,1,0,0,0,0,0,0,0,0,0
21083,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
21084,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


In [523]:
from sklearn.model_selection import train_test_split
import pandas as pd
df = pd.read_csv('data/kc_house_data_clean_knn.csv')
y = df['price']
X = df.drop(['price',  'condition'], axis=1)
Z = X.drop(['waterfront' , 'grade'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.04 , random_state=42)


# On cree un pipeline de proprocessing pour les variables numériques et une autre pour les variables catégorielles
numeric_features = Z.columns # on récupère les noms des variables numériques
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from sklearn.decomposition import PCA

from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')), # imputation des valeurs manquantes
        ('standard', StandardScaler()), # standardisation des données
        ('min_max', MinMaxScaler()) # normalisation des données
        ])



# a l'aide de la classe ColumnTransformer, 
categorical_features = ['waterfront' , 'grade'] # on récupère les noms des variables catégorielles

from sklearn.preprocessing import OneHotEncoder
categorical_transformer = OneHotEncoder(sparse=True)

# on déclare à quelles variables on applique quel transformer
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


#On obtient un pipeline de preprocessing qu'on peut utiliser dans un pipeline d'entainement
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
knn_4 = KNeighborsRegressor(n_neighbors=10 , weights='distance', p=1)
pipe = Pipeline([
     ('prep', preprocessor),
     ('knn', knn_4)
])

In [524]:
# Entrainement sur X_train
trained_pipe = pipe.fit(X_train,y_train)

# scoring sur X_test
trained_pipe.score(X_test,y_test)

/home/thomas59/anaconda3/envs/machine-learning/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.8632521292768666

In [525]:
predict_lr_X = trained_pipe.predict(X_test)
df_model = X_test.copy()
df_model['price'] = y_test
df_model['predict_price'] = predict_lr_X.round(0)
df_model = df_model.iloc[:len(df_model)]
df_model[['price', 'predict_price']].head(60)

,price,predict_price
15427,277500.0,284372.0
5076,492450.0,487124.0
5537,516130.0,521707.0
6508,414999.0,463699.0
10574,625000.0,607905.0
7459,575000.0,567441.0
14297,385000.0,502271.0
3798,418000.0,373176.0
13804,426000.0,354748.0
9838,430000.0,441715.0


In [526]:
Z

,bedrooms,bathrooms,surface,sqft_lot,floors,view,sqft_above,sqft_basement,sqft_living15,sqft_lot15,...,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199
0,3,1.00,1180,5650,1.0,0,1180,0,1340,5650,...,0,0,0,0,0,0,1,0,0,0
1,3,2.25,2570,7242,2.0,0,2170,400,1690,7639,...,0,0,0,0,0,0,0,0,0,0
2,2,1.00,770,10000,1.0,0,770,0,2720,8062,...,0,0,0,0,0,0,0,0,0,0
3,4,3.00,1960,5000,1.0,0,1050,910,1360,5000,...,0,0,0,0,0,0,0,0,0,0
4,3,2.00,1680,8080,1.0,0,1680,0,1800,7503,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21081,3,2.50,1530,1131,3.0,0,1530,0,1530,1509,...,0,0,0,0,0,0,0,0,0,0
21082,4,2.50,2310,5813,2.0,0,2310,0,1830,7200,...,1,0,0,0,0,0,0,0,0,0
21083,2,0.75,1020,1350,2.0,0,1020,0,1020,2007,...,0,0,0,0,0,0,0,0,0,0
21084,3,2.50,1600,2388,2.0,0,1600,0,1410,1287,...,0,0,0,0,0,0,0,0,0,0
